# Análise e Implementação do Algoritmo de Dijkstra
O **Algoritmo de Dijkstra**, também conhecido no contexto da Inteligência Artificial como **Busca de Custo Uniforme (Uniform Cost Search - UCS)**. Faremos uma análise teórica de suas propriedades e o compararemos com outros algoritmos de busca. Ao final, apresentaremos uma implementação em Python utilizando uma estrutura de dados avançada, o **Heap de Fibonacci**, para otimização.


## 1. Análise da Busca de Custo Uniforme (Algoritmo de Dijkstra)

A Busca de Custo Uniforme é uma estratégia de busca sem informação que pode ser implementada como uma busca de "Melhor Primeira Escolha", onde a fronteira (o conjunto de nós a serem explorados) é gerenciada por uma fila de prioridades ordenada pela função de custo do caminho acumulado ($g(n)$).

### **Propriedades do Algoritmo**

- **Completude:** **Sim**, o algoritmo é completo. Ele garante que encontrará uma solução se ela existir, desde que os custos das ações sejam maiores que um valor positivo mínimo ($\epsilon > 0$) para evitar ciclos de custo zero.
- **Entrega da Solução Ótima:** **Sim**, esta é a sua principal força. O algoritmo sempre encontra a solução de menor custo. Como ele expande os nós em ordem crescente de custo acumulado, a primeira vez que atinge um nó objetivo, é garantido que o caminho encontrado é o mais barato possível.
- **Complexidade de Tempo e Espaço:** A complexidade é de **$O(b^{1+\lfloor C^{*}/\epsilon\rfloor})$**, onde `b` é o fator de ramificação, $C^{*}$ é o custo da solução ótima e $\epsilon$ é o menor custo de ação. Isso demonstra que sua performance é sensível não apenas ao número de "passos", mas também ao custo total do caminho. Com implementações otimizadas (como heaps), a complexidade de tempo é frequentemente descrita como $O(E + V \log V)$, onde `V` são os vértices e `E` as arestas.


## 2. Comparativo: Busca de Custo Uniforme vs. Busca em Largura

| Característica | Busca de Custo Uniforme (UCS / Dijkstra) | Busca em Largura (BFS) |
| :--- | :--- | :--- |
| **Critério de Expansão** | Expande o nó com o menor **custo de caminho acumulado** ($g(n)$). | Expande o nó na **menor profundidade** (menor número de ações). |
| **Estrutura de Dados**| Utiliza uma **fila de prioridade** (como um heap) ordenada pelo custo. | Utiliza uma **fila comum (FIFO)**. |
| **Garantia de Otimização** | **Sempre** encontra a solução de menor custo (para custos não negativos). | Só garante a solução de menor custo se **todas as ações tiverem custos idênticos**. |
| **Diferença Fundamental** | É uma generalização da BFS, projetada para lidar com custos de ação variáveis. | Pode ser vista como um caso especial da UCS, onde todos os custos de ação são iguais a 1. |


## 3. Implementação com Heap de Fibonacci

A eficiência do Algoritmo de Dijkstra depende fortemente da estrutura de dados usada para a fila de prioridade. Um **Heap de Fibonacci** é uma estrutura de dados que oferece um desempenho amortizado mais rápido para a operação de "diminuir a chave" (essencial quando encontramos um caminho mais curto para um nó que já está na fronteira). Isso torna a implementação geral mais rápida, especialmente em grafos densos.


In [ ]:
pip install fibheap # Não tava funcionando 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import math
import heapq

def dijkstra_fibonacci_heap(graph, start_node):
    """
    Implementação prática do Dijkstra usando heapq (min-heap binário).
    A complexidade é O((V + E) log V), similar à versão com Fibonacci Heap
    para grafos de tamanho moderado.
    """
    
    distances = {node: math.inf for node in graph}
    distances[start_node] = 0
    previous_nodes = {node: None for node in graph}
    
    pq = [(0, start_node)]  # (distância, nó)
    
    while pq:
        current_dist, current_node = heapq.heappop(pq)
        
        if current_dist > distances[current_node]:
            continue
        
        for neighbor, weight in graph[current_node].items():
            new_dist = current_dist + weight
            
            if new_dist < distances[neighbor]:
                distances[neighbor] = new_dist
                previous_nodes[neighbor] = current_node
                heapq.heappush(pq, (new_dist, neighbor))
    
    return distances, previous_nodes


def reconstruct_path(previous_nodes, start_node, end_node):
    path = []
    current = end_node
    while current is not None:
        path.insert(0, current)
        current = previous_nodes.get(current)
    return path if path and path[0] == start_node else None

In [2]:
# --- Teste com o grafo da Romênia ---
romania_graph = {
    'Arad': {'Zerind': 75, 'Sibiu': 140, 'Timisoara': 118},
    'Zerind': {'Arad': 75, 'Oradea': 71},
    'Oradea': {'Zerind': 71, 'Sibiu': 151},
    'Sibiu': {'Arad': 140, 'Oradea': 151, 'Fagaras': 99, 'Rimnicu Vilcea': 80},
    'Timisoara': {'Arad': 118, 'Lugoj': 111},
    'Lugoj': {'Timisoara': 111, 'Mehadia': 70},
    'Mehadia': {'Lugoj': 70, 'Drobeta': 75},
    'Drobeta': {'Mehadia': 75, 'Craiova': 120},
    'Craiova': {'Drobeta': 120, 'Rimnicu Vilcea': 146, 'Pitesti': 138},
    'Rimnicu Vilcea': {'Sibiu': 80, 'Craiova': 146, 'Pitesti': 97},
    'Fagaras': {'Sibiu': 99, 'Bucharest': 211},
    'Pitesti': {'Rimnicu Vilcea': 97, 'Craiova': 138, 'Bucharest': 101},
    'Bucharest': {'Fagaras': 211, 'Pitesti': 101, 'Giurgiu': 90, 'Urziceni': 85},
    'Giurgiu': {'Bucharest': 90},
    'Urziceni': {'Bucharest': 85, 'Hirsova': 98, 'Vaslui': 142},
    'Hirsova': {'Urziceni': 98, 'Eforie': 86},
    'Eforie': {'Hirsova': 86},
    'Vaslui': {'Urziceni': 142, 'Iasi': 92},
    'Iasi': {'Vaslui': 92, 'Neamt': 87},
    'Neamt': {'Iasi': 87}
}

start = 'Arad'
end = 'Bucharest'

distances, previous = dijkstra_fibonacci_heap(romania_graph, start)
path = reconstruct_path(previous, start, end)

print(f"Análise do caminho de '{start}' para '{end}':\n")
if path:
    print(f"  -> Custo total do caminho mais curto: {distances[end]}")
    print(f"  -> Caminho: {' -> '.join(path)}")
else:
    print("Não foi possível encontrar um caminho.")

Análise do caminho de 'Arad' para 'Bucharest':

  -> Custo total do caminho mais curto: 418
  -> Caminho: Arad -> Sibiu -> Rimnicu Vilcea -> Pitesti -> Bucharest
